In [16]:
import psycopg2
from psycopg2 import Error

try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",
                                  password="Mac.phil.007",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="Platform")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)


PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'Platform', 'host': '127.0.0.1', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslnegotiation': 'postgres', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 



# Fecth from Apify 

HTTPS method

In [17]:
import http.client

# Initialize connection to Apify
conn = http.client.HTTPSConnection("api.apify.com")

# Set headers to request CSV format
headers = {
    'Accept': 'application/csv'
}

# Make the request (Replace with your dataset ID)
conn.request("GET", "/v2/datasets/txGdzlgXHqvY0fXdC/items?format=csv", headers=headers)

# Get response
res = conn.getresponse()
data = res.read()

# Save data as CSV file
csv_filename = "polymarket_data1000.csv"
with open(csv_filename, "wb") as f:
    f.write(data)

print(f"✅ CSV file saved as {csv_filename}")


✅ CSV file saved as polymarket_data1000.csv


In [18]:
import pandas as pd

# Load CSV into DataFrame
csv_filename = "polymarket_data1000.csv"
df1000 = pd.read_csv(csv_filename)

# Display the first few rows
df1000.shape[0]


1000

# Insert into pgsql

- Don't forget to create the table first in pgsql

Create the table in pgsql

In [19]:
# # Rollback any failed transactions
# connection.rollback()

# # Drop the table if it already exists
# cursor.execute("DROP TABLE IF EXISTS polymarket_data1000;")
# connection.commit()

# # Create a new table
# create_table_query = '''CREATE TABLE polymarket_data1000 (
#     id SERIAL PRIMARY KEY,
#     market_chance FLOAT8,
#     market_closed BOOLEAN,
#     market_creation_date TIMESTAMP,
#     market_description TEXT,
#     market_end_date TIMESTAMP,
#     market_link TEXT,
#     market_liquidity FLOAT8,
#     market_no_price FLOAT8,
#     market_num_comments INT,
#     market_one_day_price_change FLOAT8,
#     market_question TEXT,
#     market_rewards_amount FLOAT8,
#     market_rewards_daily_rate FLOAT8,
#     market_spread FLOAT8,
#     market_volume FLOAT8,
#     market_volume_24 FLOAT8,
#     market_yes_price FLOAT8
# );'''

# cursor.execute(create_table_query)
# connection.commit()
# print("✅ Table dropped and recreated successfully!")


In [20]:
connection.rollback()
import pandas as pd

# Assuming df is your DataFrame
df1000['market_creation_date'] = df1000['market_creation_date'].apply(lambda x: None if pd.isna(x) else x)
df1000['market_end_date'] = df1000['market_end_date'].apply(lambda x: None if pd.isna(x) else x)

# Now insert data into PostgreSQL
for _, market in df1000.iterrows():
    cursor.execute("""
        INSERT INTO polymarket_data (
            market_question, market_description, market_creation_date, market_end_date, 
            market_chance, market_liquidity, market_volume, market_volume_24, 
            market_yes_price, market_no_price, market_spread, 
            market_rewards_amount, market_rewards_daily_rate, market_num_comments, market_link
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (


    ))

connection.commit()
print("✅ Scraped Polymarket data stored in PostgreSQL successfully!")


✅ Scraped Polymarket data stored in PostgreSQL successfully!
